# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La recuperación de información se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogeneas o heterogeneas, tanto en su contenido como en su formato. En su inicio, se consideraban colecciones unicamente de texto pero las necesidades de información se han diversificado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imagenes o videos, o include multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogeneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolecados, aunque se invita a usar sus propias colecciones de tal forma que puedan sacar provecho de los temas de manera más amplia. La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.


# Ejemplos de sistemas de recuperación información

- Google,scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo
- ...

También es común que el sistema de recuperación de información no sea el principal producto, si no más bien un complemento imperdible
como es el caso de los sistemas de streaming de videos, películas, música, redes sociales, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejemplo


Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

Nota: La primera vez que se corra este notebook los paquetes necesarios se instalaran de manera automática, es posible que tome un tiempo considerable.



In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, LinearAlgebra, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    voc = Vocabulary(textconfig, corpus)
    model = VectorModel(IdfWeighting(), TfWeighting(), voc)    
    vectors = vectorize_corpus(model, corpus)

    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, _ = read_emojispace()
T = text_model_and_vectors(D.text)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})

### Note que las etiquetas son emojis, que hay 45k ejemplos y un vocabulario de más de 42k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(T.model.voc))
append_items!(invfile, VectorDatabase(T.vectors))

{WeightedInvertedFile{Nothing, SimilaritySearch.AdjacencyLists.AdjacencyList{IdWeight}} vocsize=41704, n=45000}

In [5]:
E = (; invfile, data=D, T...); # a convenient definition to put everything into a single tuple

In [6]:
function search_and_display(E, q, k)
    res = KnnResult(k)
    @time search(E.invfile, vectorize(E.model, q), res)
    L = []
    for (i, p) in enumerate(res)
        id, dist = p.id, round(p.weight, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(E.data.labels[id])</td> <td>$(E.data.text[id])</td> </tr>""")
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [7]:
search_and_display(E, "no importa distancia", 5)
search_and_display(E, "cuando nos vemos?", 5)
search_and_display(E, "feliz cumpleaños?", 3)
search_and_display(E, "ola k ase", 3)
search_and_display(E, "mi humilde opinión", 10)

  0.025915 seconds (1.21 k allocations: 56.797 KiB, 98.89% compilation time)


1,35343 => 0.03,💕,No importa la distancia _emo _usr _url
2,24815 => 0.38,😓,Me caga la distancia _emo
3,37163 => 0.52,😃,No importa cuando... _emo _emo _url
4,1288 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"
5,36425 => 0.57,😑,_usr A ti no te importa _emo


  0.000252 seconds (25 allocations: 3.602 KiB)


1,30054 => 0.23,😂,Nos vemos!! _emo _emo _emo _emo _url
2,21749 => 0.31,😜,_usr En la tarde nos vemos _emo _emo _emo
3,9076 => 0.34,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,24169 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo
5,37158 => 0.45,😕,Apenas van _num días que no nos vemos _emo


  0.000132 seconds (23 allocations: 3.547 KiB)


1,26556 => 0.11,❤,_usr Feliz Cumpleaños _emo
2,40229 => 0.11,😁,Feliz Cumpleaños _emo _emo _usr
3,8772 => 0.11,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000022 seconds (22 allocations: 3.406 KiB)


1,43663 => 0.64,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,23585 => 0.71,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,38425 => 0.71,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000152 seconds (23 allocations: 3.523 KiB)


1,2148 => 0.33,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,35818 => 0.51,😍,_usr es la persona más humilde que conozco _emo
3,32858 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,28393 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,31667 => 0.65,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,5532 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,12713 => 0.67,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,990 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
9,39601 => 0.73,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"
10,11589 => 0.75,😃,Interesante opinión y espero que sigas escribiendo _usr en mejores espacios !suerte!! _emo _emo _emo _url


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [8]:
for i in 1:10
    qID = rand(1:length(E.data.text))
    search_and_display(E, E.data.text[qID], 7)
end

  0.004698 seconds (58 allocations: 5.453 KiB)


1,41834 => 0.0,🙊,"_usr me cortare el cabello así chiquito, por fin me saldrán tus peinados _emo"
2,44906 => 0.72,🤤,Chiquito bebé _emo _emo _emo _emo _emo _url
3,36549 => 0.72,😣,Se me hizo chiquito el corazón _emo
4,3450 => 0.74,😏,_usr llego el domingo chiquito _emo
5,37009 => 0.75,👌,"_usr _usr Y bonita mamá, aún recuerdo que vi tus bellos peinados y pompitas en Oax. _emo _emo"
6,14627 => 0.75,😌,Me urge algo así en mi cabello _emo _emo _emo _emo _url
7,19394 => 0.76,🙊,Amo mi cabello. _emo _url


  0.005528 seconds (58 allocations: 5.453 KiB)


1,43818 => 0.0,💜,"_usr Siempre estaré contigo, espero que te haiga gustado _emo _emo #M_num _url"
2,21235 => 0.67,😍,_usr Ahí estaré siempre _emo
3,28052 => 0.68,😎,Monterrey siempre me ha gustado... _emo _emo _emo _emo
4,40492 => 0.68,🙈,Tengo el mejor novio del mundo #M _emo _emo
5,21585 => 0.71,🙊,"""A pesar de todo lo que pase, siempre estaré contigo"" _emo _emo _usr _url"
6,32897 => 0.72,😏,Me hubiera gustado ir _emo _emo _emo
7,14004 => 0.72,😓,Me hubiera gustado ir _emo _emo


  0.001400 seconds (26 allocations: 3.664 KiB)


...

1,12889 => -0.0,❤,Con las personas que están conmigo ahorita no necesito a mas _emo _emo
2,21132 => 0.6,😩,Ya necesito más _emo _url
3,28411 => 0.61,😐,"Me encanta que cuando más necesito de mis amigos, no están. _emo"
4,38150 => 0.61,😴,Yo ahorita _emo _emo _url
5,41605 => 0.62,😩,Necesito _emo _emo _emo _emo _emo _emo _emo _emo _emo _emo
6,4903 => 0.62,😔,Yo ahorita. _emo _url
7,23116 => 0.63,😔,Mami te necesito aquí conmigo _emo _emo


  0.002407 seconds (31 allocations: 3.883 KiB)


1,23385 => 0.0,😓,Quiero irme de aquí. _emo
2,35002 => 0.12,😱,Ya quiero irme de aquí !!! _emo _emo _emo _emo
3,20820 => 0.33,😫,"No quiero irme de México, quiero quedarme aquí x siempre _emo"
4,200 => 0.45,😔,Yo no queria irme _emo
5,28753 => 0.49,😥,Cómo quisiera irme de mi casa. _emo
6,13869 => 0.5,💔,Solo quiero irme a mi casa y no saber nada de nadie. _emo
7,18482 => 0.54,😣,Ya me quiero ir de aquí _emo


  0.007927 seconds (84 allocations: 6.750 KiB)


1,7729 => 0.0,😐,"Acabo de ver una carrera de bebés gateando. El que ganó, llegó primero porque la mamá lo atraía con el _emo _emo"
2,20665 => 0.71,💖,Bebés _emo _emo
3,20173 => 0.73,🤔,_usr Y qué gano? _emo
4,13650 => 0.74,😋,"._usr No ganó el Estado de México, pero se ganó mi corazón. _emo"
5,10213 => 0.76,😋,Siempre le gano _emo
6,8519 => 0.76,😱,Llegó el día _emo
7,37905 => 0.76,❤,Amo a los bebes _emo _emo


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.